# Word2Vec for Tamil words. 

Converting words to vector representation and similarity map generation

Fine plural form of words


In [2]:
import pandas as pd
import numpy as np 
import re
from matplotlib import pyplot as plt
import regex
from tqdm import tqdm


In [3]:
import sys

In [4]:
sys.setrecursionlimit(2500)

In [5]:
In_Colab = False
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    In_Colab = True

In [6]:
if In_Colab:
    from google.colab import drive
    drive.mount('/gdrive')

    %cd /gdrive/My\ Drive/Tamizh/data/wiki_data

In [7]:
from constants import *

In [8]:
data = pd.read_csv("../data/tawiki_data_df.csv")

In [9]:
# Shape of the data
data.shape

(235997, 1)

In [10]:
data = data.dropna()

In [11]:
data.shape

(201665, 1)

#### Convert each sentence to list of tokens. 

*observation : this have more then 2 crore vocabulary 
* Need to find the root words  - 

In [12]:
data.head()

,Content
0,விக்கிப்பீடியா மொழிகள்
1,இங்கு தமிழ் விக்கிப்பீடியாவைப் பற்றிய உங்கள் ப...
2,விக்கிப்பீடியா அதன் வாசகர்களால் ஒருமித்து எழுத...
3,விக்கிப்பீடியாவுக்கு கட்டுரை எழுதி மேம்படுத்தி...
4,முகலாயக் கட்டிடக்கலைமுகலாய கட்டடக்கலையின் சிற...


### Drop NA values

### Convert as one long sentence. This will be useful for cleaning vallina migu sorrkal.

## Data Cleaning 

In [13]:
def split_text(text):
    return regex.findall(r'\X', text, regex.U)

def conv_base(text):
    return [split_map.get(k) for k in split_text(text)]

def _lastmap(text):
    if len(text[-1])>1:
        return text[-1][-1]
    return text[-1][0]

def _firstmap(text):
    return text[0][0]

def is_vallmigu_pair(text1,text2):
    try:
        text1 = conv_base(text1)
        text2 = conv_base(text2)
        if (_lastmap(text1) in vall_migu) and (_firstmap(text2) in vall_migu):
            return True
        else:
            return False
    except:
        return False

In [14]:
def clean_split(text):
    text = re.sub("\s+"," ",text.strip())
    return text.split(' ')

In [15]:
data.head()

,Content
0,விக்கிப்பீடியா மொழிகள்
1,இங்கு தமிழ் விக்கிப்பீடியாவைப் பற்றிய உங்கள் ப...
2,விக்கிப்பீடியா அதன் வாசகர்களால் ஒருமித்து எழுத...
3,விக்கிப்பீடியாவுக்கு கட்டுரை எழுதி மேம்படுத்தி...
4,முகலாயக் கட்டிடக்கலைமுகலாய கட்டடக்கலையின் சிற...


In [16]:
vallmigu_pair_set = set()

In [17]:
data['processed'] = data.Content.apply(clean_split)

In [18]:
def parse_clean_text(text):
    global vallmigu_pair_set
    for i in range(len(text)-2):
        if is_vallmigu_pair(text[i],text[i+1]):
            vallmigu_pair_set.add((text[i],text[i+1]))
            text[i] = ''.join(split_text(text[i])[:-1])
    return text

In [19]:
# from multiprocessing import Pool 

In [20]:
# with Pool(5) as p:
#     text_list = p.map(parse_clean_text,data.Content.values)

In [21]:

for row in tqdm(range(data.shape[0])):
    text = data.processed.iloc[row]
    for i in range(len(text)-2):
        if is_vallmigu_pair(text[i],text[i+1]):
            vallmigu_pair_set.add((text[i],text[i+1]))
            text[i] = ''.join(split_text(text[i])[:-1])
    data.processed.iloc[row]= text



100%|██████████| 201665/201665 [19:58<00:00, 168.20it/s]


In [31]:
# ~1200 sec nearly 20 mins 

In [22]:
data.head()

,Content,processed
0,விக்கிப்பீடியா மொழிகள்,"[விக்கிப்பீடியா, மொழிகள்]"
1,இங்கு தமிழ் விக்கிப்பீடியாவைப் பற்றிய உங்கள் ப...,"[இங்கு, தமிழ், விக்கிப்பீடியாவை, பற்றிய, உங்கள..."
2,விக்கிப்பீடியா அதன் வாசகர்களால் ஒருமித்து எழுத...,"[விக்கிப்பீடியா, அதன், வாசகர்களால், ஒருமித்து,..."
3,விக்கிப்பீடியாவுக்கு கட்டுரை எழுதி மேம்படுத்தி...,"[விக்கிப்பீடியாவுக்கு, கட்டுரை, எழுதி, மேம்படு..."
4,முகலாயக் கட்டிடக்கலைமுகலாய கட்டடக்கலையின் சிற...,"[முகலாய, கட்டிடக்கலைமுகலாய, கட்டடக்கலையின், சி..."


In [23]:
len(vallmigu_pair_set)

743967

#Word2Vec

Generate word2vec representation for tokens in tamil. 



In [24]:
import pickle

with open('../data/vallmigu_pair.pkl','wb') as f:
    pickle.dump(vallmigu_pair_set,f)

In [25]:
#Generate word 2 vec

from gensim.models import Word2Vec

In [26]:
texts = data.processed.values

In [27]:
 
model = Word2Vec(texts, size=100, window=50, min_count=10, workers=6)

In [28]:
len(model.wv.vocab)

142721

In [29]:
model.wv.most_similar("என்னுடைய")

[('என்னை', 0.7970262765884399),
 ('எனக்கு', 0.7671459913253784),
 ('மகிழ்ச்சியாக', 0.7472310066223145),
 ('நான்', 0.7441911697387695),
 ('உங்களை', 0.7333188056945801),
 ('என்னிடம்', 0.7323682308197021),
 ('எங்களுக்கு', 0.729627251625061),
 ('இருங்கள்', 0.7186785340309143),
 ('என்னால்', 0.7163692116737366),
 ('நானும்', 0.7148479223251343)]

In [30]:
model.save("../model/tawiki_w2v")